# Dataset Home Credit Default Risk

Datos de clientes de la entidad Home Credit organizados en siete archivos diferentes según la información proporcionada: datos sobre la solicitud de crédito e historia crediticia (interna o de organizaciones externas reportadas en bureaus.)

Home Credit es una compañía financiera internacional que opera en múltiples países europeos y asiáticos que opera en múltiples países europeos y asiáticos.

*Objetivo del notebook*: Realizar una exploración y limpieza de los datos (EDA)

## Exploración y limpieza de los datos

### Carga de librerías

In [1]:
import pandas as pd
import numpy as np
import dtale as dt
import warnings
import os
warnings.filterwarnings('ignore')

### Carga de ficheros

In [2]:
path = "../../data/raw/"
csv_files = {
    "application_train": "DS03_application_train.csv",
    "bureau": "DS03_bureau.csv",
    "previous_application": "DS03_previous_application.csv"
}

# Load all files into a dictionary
dict_data = {}
for key, file in csv_files.items():
    dict_data[key] = pd.read_csv(os.path.join(path,file))

### Descripción dataset

In [3]:
# Function to print the shape and unique data types per dataset
def datasets_description(df, dataset_name):
    print(f"\nDataset description: {dataset_name}")
    
    # Print dataset shape
    print(f"\nShape: {df[dataset_name].shape}")
    # Print unique data types
    print(f"Unique data types:")
    print(df[dataset_name].dtypes.value_counts())

In [ ]:
for key in dict_data.keys():
    datasets_description(dict_data, key)

### Verificación de calidad

Se ejecutan las siguientes evaluaciones. 
1. Dimensión consistencia: se evaluará la consistencia de claves de los datasets bureau y previous_application con respecto a application_train
2. Dimensión de completitud: 
    - 2.1. Evaluación de valores nulos columnas
    - 2.2. Evaluación de valores nulos filas
3. Dimensión exactitud:
    - 3.1. Evaluación de valores ajustados para atributos de tipo object
    - 3.2. Evaluación de valores ajustados para atributos de tipo numéric (int y float)



#### 1. Dimensión de consistencia

Comprobaremos que las claves SK_ID_CURR presentes en bureau y previous application existen en application_train

In [4]:
def ids_consistency(df1, df2, id_column):
    # Create sets with unique keys from both dfs
    ids_df1 = set(df1[id_column].unique())
    ids_df2 = set(df2[id_column].unique())
    
    # Find SK_ID_CURR in df1 that are not in df2
    not_matching_ids=list(set(ids_df1) - set(ids_df2))
 
    return len(not_matching_ids)

for dataset_name in ['bureau', 'previous_application']:
    print(f"\nDataset {dataset_name} against application_train:")
    res=ids_consistency(dict_data[dataset_name], dict_data['application_train'], 'SK_ID_CURR')
    print(f"\Count of ids present in {key} not matching ids in application_train: {res}")


Dataset bureau against application_train:
\Count of ids present in previous_application not matching ids in application_train: 42320

Dataset previous_application against application_train:
\Count of ids present in previous_application not matching ids in application_train: 47800


In [5]:
def remove_nonmatching_ids(df1, df2, id_column):
    # Remove rows from df1 where SK_ID_CURR not in df2, which is application_train.
    dict_data[dataset_name] = df1[df1[id_column].isin(df2[id_column])]
    return dict_data[dataset_name]


for dataset_name in ['bureau', 'previous_application']:
    print(f"\nRemove non-matching ids from {dataset_name}")
    dict_data[dataset_name]= remove_nonmatching_ids(dict_data[dataset_name], dict_data['application_train'], 'SK_ID_CURR')
    # Revaluate ids_consistenci
    res=ids_consistency(dict_data[dataset_name], dict_data['application_train'], 'SK_ID_CURR')
    print(f"Revaluation count of ids present in {key} not matching ids in application_train: {res}")


Remove non-matching ids from bureau
Revaluation count of ids present in previous_application not matching ids in application_train: 0

Remove non-matching ids from previous_application
Revaluation count of ids present in previous_application not matching ids in application_train: 0


#### 2. Dimensión de completitud: Evaluación de valores nulos

##### 2.1. A nivel atributos: comprobamos el % de missing values por cada atributo

In [6]:
# Function to check the percentage of missing values
def completeness_attributes(df):
    percentage_nulls = df.isnull().sum() / len(df) * 100
    cols_nulls = percentage_nulls[percentage_nulls > 0].sort_values(ascending=False) # Display only cols presenting missing values
    return cols_nulls

In [7]:
# Check missing percentage for each dataset
for key in dict_data.keys():
    print(f"\n{key} - Percentatges of null values for each attribute:")
    missing_percentage = completeness_attributes(dict_data[key])
    print(missing_percentage)


application_train - Percentatges of null values for each attribute:
COMMONAREA_MEDI             69.872297
COMMONAREA_AVG              69.872297
COMMONAREA_MODE             69.872297
NONLIVINGAPARTMENTS_MEDI    69.432963
NONLIVINGAPARTMENTS_MODE    69.432963
                              ...    
EXT_SOURCE_2                 0.214626
AMT_GOODS_PRICE              0.090403
AMT_ANNUITY                  0.003902
CNT_FAM_MEMBERS              0.000650
DAYS_LAST_PHONE_CHANGE       0.000325
Length: 67, dtype: float64

bureau - Percentatges of null values for each attribute:
AMT_ANNUITY               77.116885
AMT_CREDIT_MAX_OVERDUE    64.732738
DAYS_ENDDATE_FACT         37.170798
AMT_CREDIT_SUM_LIMIT      33.417160
AMT_CREDIT_SUM_DEBT       15.224882
DAYS_CREDIT_ENDDATE        6.080426
AMT_CREDIT_SUM             0.000205
dtype: float64

previous_application - Percentatges of null values for each attribute:
RATE_INTEREST_PRIMARY        99.661102
RATE_INTEREST_PRIVILEGED     99.661102
AMT_DOWN_PA

Se observan atributos con un porcentaje mayor al 65% de missing values. Se considera que estos atributos no cumplen el criterio de completitud y por tanto se eliminan de los datasets.

In [8]:
# Function to remove columns with more than a 65% of missing values
def drop_nulls_columns(df):
    # Drop columns with more than 65% missing values
    cols_to_drop = df.columns[df.isnull().sum() / len(df) > 0.65]
    df.drop(columns=cols_to_drop, inplace=True)
    print(f"Dropped columns: {list(cols_to_drop)}")
    return df

# Apply drop_nulls_columns to all datasets
for key in dict_data.keys():
    print(f"\nDataset {key}")
    dict_data[key] = drop_nulls_columns(dict_data[key])
    print(f"Final shape: {dict_data[key].shape}")


Dataset application_train
Dropped columns: ['OWN_CAR_AGE', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'FLOORSMIN_AVG', 'LIVINGAPARTMENTS_AVG', 'NONLIVINGAPARTMENTS_AVG', 'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'FLOORSMIN_MODE', 'LIVINGAPARTMENTS_MODE', 'NONLIVINGAPARTMENTS_MODE', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'FLOORSMIN_MEDI', 'LIVINGAPARTMENTS_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'FONDKAPREMONT_MODE']
Final shape: (307511, 105)

Dataset bureau
Dropped columns: ['AMT_ANNUITY']
Final shape: (1465325, 16)

Dataset previous_application
Dropped columns: ['RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED']
Final shape: (1413701, 35)


##### 2.2. A nivel atributos: comprobamos el % de atributos nulos por cada fila

In [9]:
def completeness_rows(df,acceptance_threshold):
    num_cols = len(df.axes[1])
    # Create new column with the proportion of missing values per row
    df['row_completeness'] = (df.isnull().sum(axis=1) / num_cols)
    # Create subset with rows having more than 30% of missing values
    rows_with_nulls = df[df['row_completeness'] >= acceptance_threshold]
    completeness_r = rows_with_nulls.shape[0]
    # Return the percentage of rows having more than 30% of missing values
    return round((completeness_r / df.shape[0]) * 100, 2)

# Rows exceeding the acceptance_threshold will be considered as invalid.
acceptance_threshold = 0.3

for key in dict_data.keys():
    print(f"\nDataset {key}")
    perc_rows_above_threshold = completeness_rows(dict_data[key], acceptance_threshold)
    print(f"Percentage of rows exceeding the accepted threshold of missing values: {perc_rows_above_threshold}%")


Dataset description: application_train
Percentage of rows exceeding the accepted threshold of missing values: 35.75%

Dataset description: bureau
Percentage of rows exceeding the accepted threshold of missing values: 0.87%

Dataset description: previous_application
Percentage of rows exceeding the accepted threshold of missing values: 19.43%


Se eliminan las filas que tienen más de un 30% de atributos nulos

In [10]:
def drop_nulls_rows(df):
    # Remove rows with more than 30% of missing values.

    # Count of rows not verifying the completeness criteria
    cnt_rows=len(df[df['row_completeness']>= acceptance_threshold])
    # Rewrite dataset removing incomplete rows
    df=df.drop(df[df['row_completeness'] > acceptance_threshold].index)
    # Removing row_completeness column 
    df.drop(columns='row_completeness', inplace=True)
    print(f"Number of rows deleted: {cnt_rows}. New shape of {key}: {df.shape}")
    return df

In [11]:
for key in dict_data.keys():
    print(f"\nDataset {key}")
    dict_data[key] = drop_nulls_rows(dict_data[key])


Dataset description: application_train
Number of rows deleted: 109926. New shape of application_train: (197585, 105)

Dataset description: bureau
Number of rows deleted: 12813. New shape of bureau: (1452512, 16)

Dataset description: previous_application
Number of rows deleted: 274687. New shape of previous_application: (1139014, 35)


#### 3. Dimensión de exactitud: Valores ajustados

##### 3.1. Analizamos las columnas de tipo object.

In [12]:
#Visualization of columns of object type
for key in dict_data.keys():
    print(f"\nDataset {key}")
    display(dict_data[key].select_dtypes(include='object').head(5))


Dataset description: application_train


,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
0,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,block of flats,"Stone, brick",No
1,Cash loans,F,N,N,Family,State servant,Higher education,Married,House / apartment,Core staff,MONDAY,School,block of flats,Block,No
6,Cash loans,F,Y,Y,Unaccompanied,Commercial associate,Higher education,Married,House / apartment,Accountants,SUNDAY,Business Entity Type 3,NaN,NaN,NaN
10,Cash loans,F,N,Y,Unaccompanied,Working,Higher education,Married,House / apartment,Core staff,SATURDAY,Medicine,NaN,NaN,NaN
12,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,Laborers,FRIDAY,Business Entity Type 2,block of flats,NaN,No



Dataset description: bureau


,CREDIT_ACTIVE,CREDIT_CURRENCY,CREDIT_TYPE
0,Closed,currency 1,Consumer credit
1,Active,currency 1,Credit card
2,Active,currency 1,Consumer credit
4,Active,currency 1,Consumer credit
5,Active,currency 1,Credit card



Dataset description: previous_application


,NAME_CONTRACT_TYPE,WEEKDAY_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,NAME_SELLER_INDUSTRY,NAME_YIELD_GROUP,PRODUCT_COMBINATION
0,Consumer loans,SATURDAY,Y,XAP,Approved,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,Connectivity,middle,POS mobile with interest
1,Cash loans,THURSDAY,Y,XNA,Approved,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,XNA,low_action,Cash X-Sell: low
2,Cash loans,TUESDAY,Y,XNA,Approved,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,XNA,high,Cash X-Sell: high
3,Cash loans,MONDAY,Y,XNA,Approved,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,XNA,middle,Cash X-Sell: middle
4,Cash loans,THURSDAY,Y,Repairs,Refused,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,XNA,high,Cash Street: high


Todas las columnas de object type son de tipo categórico. Estudiamos los posibles valores de cada una para identificar valores fuera de rango.

In [13]:
# Function to return unique values per each object column
def unique_values(df):
    unique_values = {}
    cols=list(df.select_dtypes(include='object'))
    if len(cols)!=0:
        for column in cols:
            unique_values[column] = [df[column].unique()]  # Convertir los valores únicos en una lista
        df_unique=pd.DataFrame(unique_values).T
        df_unique.columns = ['UNIQUE_VALUES']
        df_unique.index.name = 'ATTRIBUTE'
        return df_unique
    else:
        return "No columns of object type"

In [14]:
pd.set_option('display.max_colwidth', None)
for key in dict_data.keys():
    print(f"\nDataset {key}")
    display(unique_values(dict_data[key]))


Dataset application_train


,UNIQUE_VALUES
ATTRIBUTE,
NAME_CONTRACT_TYPE,"[Cash loans, Revolving loans]"
CODE_GENDER,"[M, F, XNA]"
FLAG_OWN_CAR,"[N, Y]"
FLAG_OWN_REALTY,"[Y, N]"
NAME_TYPE_SUITE,"[Unaccompanied, Family, Other_A, nan, Spouse, partner, Other_B, Children, Group of people]"
NAME_INCOME_TYPE,"[Working, State servant, Commercial associate, Pensioner, Unemployed, Student, Businessman, Maternity leave]"
NAME_EDUCATION_TYPE,"[Secondary / secondary special, Higher education, Incomplete higher, Lower secondary, Academic degree]"
NAME_FAMILY_STATUS,"[Single / not married, Married, Widow, Civil marriage, Separated, Unknown]"
NAME_HOUSING_TYPE,"[House / apartment, Rented apartment, With parents, Municipal apartment, Office apartment, Co-op apartment]"



Dataset bureau


,UNIQUE_VALUES
ATTRIBUTE,
CREDIT_ACTIVE,"[Closed, Active, Sold, Bad debt]"
CREDIT_CURRENCY,"[currency 1, currency 2, currency 4, currency 3]"
CREDIT_TYPE,"[Consumer credit, Credit card, Mortgage, Car loan, Microloan, Loan for working capital replenishment, Loan for business development, Unknown type of loan, Another type of loan, Real estate loan, Loan for the purchase of equipment, Cash loan (non-earmarked), Mobile operator loan, Interbank credit, Loan for purchase of shares (margin lending)]"



Dataset previous_application


,UNIQUE_VALUES
ATTRIBUTE,
NAME_CONTRACT_TYPE,"[Consumer loans, Cash loans, Revolving loans]"
WEEKDAY_APPR_PROCESS_START,"[SATURDAY, THURSDAY, TUESDAY, MONDAY, FRIDAY, WEDNESDAY, SUNDAY]"
FLAG_LAST_APPL_PER_CONTRACT,"[Y, N]"
NAME_CASH_LOAN_PURPOSE,"[XAP, XNA, Repairs, Everyday expenses, Building a house or an annex, Other, Purchase of electronic equipment, Medicine, Payments on other loans, Urgent needs, Buying a used car, Buying a new car, Buying a holiday home / land, Education, Buying a home, Journey, Car repairs, Furniture, Buying a garage, Business development, Hobby, Wedding / gift / holiday, Gasification / water supply, Refusal to name the goal, Money for a third person]"
NAME_CONTRACT_STATUS,"[Approved, Refused, Unused offer, Canceled]"
NAME_PAYMENT_TYPE,"[Cash through the bank, XNA, Non-cash from your account, Cashless from the account of the employer]"
CODE_REJECT_REASON,"[XAP, HC, LIMIT, SCOFR, CLIENT, SCO, XNA, VERIF, SYSTEM]"
NAME_TYPE_SUITE,"[nan, Unaccompanied, Spouse, partner, Family, Children, Other_B, Other_A, Group of people]"
NAME_CLIENT_TYPE,"[Repeater, New, Refreshed, XNA]"


Se observan que algunos atributos contienen el valor XNA. Por ejemplo:
- application_train: CODE_GENDER, ORGANIZATION_TYPE
- bureau: STATUS
- previous application: NAME_CONTRACT_TYPE

Identificamos todos los atributos que contienen XNA en una lista y reemplazaremos XNA por nulos.

In [15]:
def attributes_containing_XNA(df):
    columns_with_val = []
    
    # Iterar por las columnas y verificar si el valor está en alguna de las filas
    for column in df.columns:
        if 'XNA' in df[column].values:
            columns_with_val.append(column)
    print(f"\Columns with XNA value: {columns_with_val}.")
    df[columns_with_val]=df[columns_with_val].replace('XNA', np.nan)
    print(f"Column values replaced")
    
    return columns_with_val

In [16]:
for key in dict_data.keys():
    print(f"\nDataset {key}:")
    attributes_containing_XNA(dict_data[key])


Dataset application_train:
\Columns with XNA value: ['CODE_GENDER', 'ORGANIZATION_TYPE'].
Column values replaced

Dataset bureau:
\Columns with XNA value: [].
Column values replaced

Dataset previous_application:
\Columns with XNA value: ['NAME_CASH_LOAN_PURPOSE', 'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON', 'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE', 'NAME_SELLER_INDUSTRY', 'NAME_YIELD_GROUP'].
Column values replaced


##### 3.2. Analizamos las columnas de tipo numérico (int y float).

Para ello se utiliza la librería dtale, para hacer una exploración rápida

In [ ]:
for key in dict_data.keys():
    dtale_df = dt.show(dict_data[key])
    dtale_df.open_browser()

**Dataset application_train**

Columnas referidas a edades (DAYS_BIRTH, DAYS_EMPLOYED, DAYS_REGISTRATION, DAYS_ID_PUBLISH) se presentan con valores negativos. Aunque puede parecer un error, se entiende que se mide con respecto a la fecha de solicitud de préstamo, por tanto representa la cantidad de días previos a la solicitud. 

No obstante se observa un valor inesperado para DAYS_EMPLOYED, donde el máximo es 365.243. Este valor además de no ser negativo, como se espera, si lo medimos en años resultan ser 365243/365=1000 años, lo cual no es posible. Analizamos si existen otros valores positivos para esta columna.

In [22]:
unique_positive_values=dict_data['application_train'].loc[dict_data['application_train']['DAYS_EMPLOYED']>0]['DAYS_EMPLOYED'].unique()
qty_rows= dict_data['application_train'].loc[dict_data['application_train']['DAYS_EMPLOYED']>0]['DAYS_EMPLOYED'].count()

print(f"Los valores positivos que se encuentran para la columna DAYS_EMPLOYED son: {unique_positive_values[0]}")
print(f"Porcentaje de filas afectadas son: {round(100*qty_rows/dict_data['application_train'].shape[0],2)}%")

Los valores positivos que se encuentran para la columna DAYS_EMPLOYED son: 365243
Porcentaje de filas afectadas son: 14.21%


El valor erroneo se encuentra en el 14.21% de las filas de application_train. Se reemplaza por nulo.

In [23]:
dict_data['application_train']['DAYS_EMPLOYED']=dict_data['application_train']['DAYS_EMPLOYED'].replace(365243, np.nan)

Todos los campos indicando flags (FLAG_MOBIL, FLAG_EMP_PHONE, FLAG_WORK_PHONE, FLAG_CONT_MOBILE,FLAG_PHONE, FLAG_EMAIL, FLAG_DOCUMENT_xx ) contienen solo valores 0 o 1. Se valida el contenido.

CNT_CHILDREN es entero y positivo. 
CNT_FAM_MEMBERS está registrado como float aunque se entiende que debe ser entero. Estudiamos si existe algún valor decimal. De lo contrario transformamos la columna a entero.

In [25]:
dict_data['application_train']['CNT_FAM_MEMBERS'].unique()

array([ 1.,  2.,  3.,  4.,  5.,  6.,  9.,  8., 10.,  7., 13., 14., 12.,
       15., nan, 20., 16.])

Todos son valores enteros, se transforma la columna a entero

In [26]:
dict_data['application_train']['CNT_FAM_MEMBERS'] = dict_data['application_train']['CNT_FAM_MEMBERS'].astype('Int64')

**Dataset PREVIOUS_APPLICATION**

- AMT_ANNUITY, AMT_APPLICAITON, AMT_CREDIT contienen valores positivos, como se espera. 
- CNT_PAYMENT es float pero se entiende que debe ser entero. Realizamos la misma comprobación que para CNT_FAM_MEMBERS de application_train

In [27]:
dict_data['previous_application']['CNT_PAYMENT'].unique() 

array([12., 36., 24., 18., 54., 30.,  6.,  8.,  0., 10., 60., 48., nan,
       42.,  5.,  4., 16., 20.,  7., 14.,  9., 22.,  3., 15., 11., 13.,
       17., 26., 23., 38., 33., 72., 59., 34., 84., 28., 41., 66., 29.,
       32., 44., 39., 40., 35., 19., 47., 45., 21., 53.])

Como todos los valores son enteros, se transforma la columna CNT_PAYMENT a entero

In [28]:
dict_data['previous_application']['CNT_PAYMENT'] = dict_data['previous_application']['CNT_PAYMENT'].astype('Int64')

Se estudian los campos que contienen días y por tanto, se espera que sean valores negativos puesto que se miden con respecto al pasado

In [29]:
for col in ['DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION']:
    unique_values=list(dict_data['previous_application'].loc[dict_data['previous_application'][col]>0][col].unique())
    print(f"\nColumn {col} unique values higher than 0: {sorted(unique_values,reverse=True)}")


Column DAYS_FIRST_DRAWING unique values higher than 0: [365243.0]

Column DAYS_FIRST_DUE unique values higher than 0: [365243.0]

Column DAYS_LAST_DUE_1ST_VERSION unique values higher than 0: [365243.0, 2389.0, 2098.0, 2090.0, 2032.0, 2016.0, 2011.0, 1993.0, 1990.0, 1954.0, 1947.0, 1941.0, 1800.0, 1798.0, 1797.0, 1795.0, 1793.0, 1792.0, 1791.0, 1790.0, 1788.0, 1785.0, 1784.0, 1783.0, 1782.0, 1781.0, 1780.0, 1779.0, 1778.0, 1777.0, 1776.0, 1774.0, 1773.0, 1772.0, 1771.0, 1770.0, 1769.0, 1768.0, 1767.0, 1766.0, 1765.0, 1764.0, 1763.0, 1762.0, 1760.0, 1759.0, 1758.0, 1757.0, 1756.0, 1755.0, 1753.0, 1752.0, 1751.0, 1750.0, 1749.0, 1748.0, 1747.0, 1746.0, 1745.0, 1744.0, 1743.0, 1742.0, 1741.0, 1740.0, 1739.0, 1738.0, 1737.0, 1736.0, 1735.0, 1734.0, 1733.0, 1731.0, 1730.0, 1729.0, 1728.0, 1727.0, 1726.0, 1725.0, 1724.0, 1723.0, 1722.0, 1721.0, 1720.0, 1719.0, 1718.0, 1717.0, 1716.0, 1715.0, 1714.0, 1713.0, 1712.0, 1711.0, 1710.0, 1709.0, 1708.0, 1707.0, 1706.0, 1705.0, 1704.0, 1703.0, 1702

Se encuentra el valor 365.243, analogo a DAYS_EMPLOYED de application_train. Es siguiente valor mayor, 2389 días corresponde a 6 años, de manera que es un valor posible. Se considera que el valor 365243 es erróneo, por tanto se reemplaza por nulo.

In [30]:
for col in ['DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION']:
    cnt=dict_data['previous_application'].loc[dict_data['previous_application'][col]>0][col].count()
    print(f"\nNumber of rows where column {col} is 365243: {cnt}")
    dict_data['previous_application'][col]=dict_data['previous_application'][col].replace(365243, np.nan)
    print(f"Value replaced for null in column {col}")


Number of rows where column DAYS_FIRST_DRAWING is 365243: 799094
Value replaced for null in column DAYS_FIRST_DRAWING

Number of rows where column DAYS_FIRST_DUE is 365243: 33962
Value replaced for null in column DAYS_FIRST_DUE

Number of rows where column DAYS_LAST_DUE_1ST_VERSION is 365243: 273637
Value replaced for null in column DAYS_LAST_DUE_1ST_VERSION

Number of rows where column DAYS_LAST_DUE is 365243: 180792
Value replaced for null in column DAYS_LAST_DUE

Number of rows where column DAYS_TERMINATION is 365243: 194000
Value replaced for null in column DAYS_TERMINATION


**Dataset bureau**

- Columnas que representan días: DAYS_CREDIT negativos, CREDIT_DAY_OVERDUE positivo, DAYS_CREDIT_ENDDATE positivo y negativo (tiene sentido refiriéndose a los creditos ya vencidos o que están por vencer), DAYS_ENDDATE_FACT negativos. Se valida el contenido
- CNT_credit_prolong: valores positivos
- AMT_CREDIT_SUM: positivos y max 396,000.
- AMT_CREDIT_SUM_LIMIT: positivos y negativos
- AMT_CREDIT_SUM_DEBT: tiene valores negativos y positivos. Consideramos que los negativos son erróneos.
- AMT_CREDIT_SUM_OVERDUE: positivos

Se estudia la proporción de registros que presentan valores negativos en AML_CREDIT_SUM_DEBT

In [35]:
# Calculate percentage of rows with negative values AML_CREDIT_SUM_DEBT
num_rows=dict_data['bureau'].shape[0]
neg_rows=len(dict_data['bureau'].loc[dict_data['bureau']['AMT_CREDIT_SUM_DEBT']<0])
perc_negative=round(100*neg_rows/num_rows,2)
print(f"La columna AML_CREDIT_SUM_DEBT presenta {perc_negative}% de resigstros con valores negativos")

La columna AML_CREDIT_SUM_DEBT presenta 0.58% de resigstros con valores negativos


La proporción de valores negativos en AML_CREDIT_SUM_DEBT es muy baja. Reemplazaremos los valores negativos de deuda por 0.

In [43]:
dict_data['bureau']['AMT_CREDIT_SUM_DEBT']=dict_data['bureau']['AMT_CREDIT_SUM_DEBT'].clip(lower=0)

#### 4. Revaluación de valores nulos tras las operaciones realizadas por la dimensión de exactitud

Completitud de columnas

In [44]:
# For columns
for key in dict_data.keys():
    print(f"\n{key} - Percentatges of null values for each attribute:")
    missing_percentage = completeness_attributes(dict_data[key])
    print(missing_percentage)


application_train - Percentatges of null values for each attribute:
EXT_SOURCE_1                    44.161753
LANDAREA_MODE                   36.776071
LANDAREA_MEDI                   36.776071
LANDAREA_AVG                    36.776071
BASEMENTAREA_MEDI               35.437407
BASEMENTAREA_MODE               35.437407
BASEMENTAREA_AVG                35.437407
NONLIVINGAREA_MEDI              30.243693
NONLIVINGAREA_MODE              30.243693
NONLIVINGAREA_AVG               30.243693
ELEVATORS_AVG                   27.313308
ELEVATORS_MEDI                  27.313308
ELEVATORS_MODE                  27.313308
OCCUPATION_TYPE                 25.358706
WALLSMATERIAL_MODE              23.496217
APARTMENTS_MODE                 23.355518
APARTMENTS_MEDI                 23.355518
APARTMENTS_AVG                  23.355518
ENTRANCES_MODE                  22.725915
ENTRANCES_MEDI                  22.725915
ENTRANCES_AVG                   22.725915
LIVINGAREA_MODE                 22.518916
LIVINGA

La única columna que supera el 65% de nulos es DAYS_FIRST_DRAWING del dataset previous_application. La eliminamos.

In [45]:
# Apply drop_nulls_columns to all datasets
for key in dict_data.keys():
    print(f"\nDataset {key}")
    dict_data[key] = drop_nulls_columns(dict_data[key])
    print(f"Final shape: {dict_data[key].shape}")


Dataset application_train
Dropped columns: []
Final shape: (197585, 105)

Dataset bureau
Dropped columns: []
Final shape: (1452512, 16)

Dataset previous_application
Dropped columns: ['DAYS_FIRST_DRAWING']
Final shape: (1139014, 34)


Completitud de filas

In [46]:
# For rows
for key in dict_data.keys():
    print(f"\nDataset description: {key}")
    perc_rows_above_threshold = completeness_rows(dict_data[key], acceptance_threshold)
    print(f"Percentage of rows exceeding the accepted threshold of missing values: {perc_rows_above_threshold}%")


Dataset description: application_train
Percentage of rows exceeding the accepted threshold of missing values: 0.12%

Dataset description: bureau
Percentage of rows exceeding the accepted threshold of missing values: 0.0%

Dataset description: previous_application
Percentage of rows exceeding the accepted threshold of missing values: 10.85%


Eliminamos las filas que no cumplen el criterio del dataset application_train pues para los otros datasets se crearán nuevos atributos y se evaluará si los nulos se presentan en las columnas relevantes.

In [47]:
dict_data['application_train'] = drop_nulls_rows(dict_data['application_train'])

Number of rows deleted: 238. New shape of previous_application: (197347, 105)


### Guardado de datasets en Preprocessed

In [52]:
for key in dict_data.keys():
    dict_data[key].to_csv("../../data/preprocessed/"+str(key)+"_prep.csv")

Añadimos los csv preprocesados a dvc